In [931]:
from datetime import datetime, timezone
from typing import List, Dict, Tuple
from dataclasses import dataclass
from os.path import join
import numpy as np
import pandas as pd
from pynwb import NWBFile, NWBHDF5IO, TimeSeries
from pynwb.file import Subject
from pynwb.ecephys import ElectricalSeries, ElectrodeGroup, SpikeEventSeries, LFP, EventWaveform
from pynwb.behavior import BehavioralEvents
import nixio
import regex as re

In [932]:
@dataclass(frozen=True)
class SessionContext:
    subject: int
    session: int
    nix: nixio.File
    nwb: NWBFile

In [933]:
def main() -> SessionContext:
    ctx = create_context(1, 1)
    write_subject(ctx)
    write_eeg_electrodes(ctx)
    write_eeg_measurements(ctx)
    ieeg_electrode_group = write_ieeg_electrodes(ctx)
    write_ieeg_measurements(ctx)
    write_trial_data(ctx)
    write_behavior(ctx)
    write_events(ctx)
    write_waveforms(ctx, ieeg_electrode_group)
    # write_nwb(ctx)
    return ctx

In [934]:
def create_context(subject: int, session: int) -> SessionContext:
    nix = _read_nix(subject, session)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description=_get_task(nix),
        identifier=f"Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject{subject:02}_session{session:02}",
        session_start_time=datetime(2019, 3, 27, 12, 00, tzinfo=timezone.utc),  # TODO: Get this from Johannes
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        related_publications=_get_related_publications(nix)
    )
    return SessionContext(subject, session, nix, nwb)


In [935]:
def _read_nix(subject: int, session: int) -> nixio.File:
    _nix_dir = join("/", "mnt", "c", "Users", "conta", "git", "USZ_NCH", "Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
                    "data_nix")
    _file_path = join(_nix_dir, f"Data_Subject_{subject:02}_Session_{session:02}.h5")
    return nixio.File.open(_file_path, nixio.FileMode.ReadOnly)

In [936]:
def _get_task(nix: nixio.File) -> str:
    task = nix.sections["Task"].props
    task_name = task["Task name"].values[0]
    # Broken UTF-8 in file
    task_desc = "The task is a modified Sternberg task in which the encoding of memory items, maintenance, and recall were temporally separated. Each trial starts with a fixation period ([-6, -5] s), followed by the stimulus ([-5, -3] s). The stimulus consists of a set of eight consonants at the center of the screen. The middle four, six, or eight letters are the memory items,which determine the set size for the trial (4, 6, or 8, respectively). The outer positions are filled with “X,” which is never a memory item. After the stimulus, the letters disappear from the screen, and the maintenance interval starts ([-3, 0] s).A fixation square is shown throughout fixation, encoding, and maintenance. After maintenance, a probe letter is presented. The subjects respond with a button press to indicate whether the probe was part of the stimulus.The subjects are instructed to respond as rapidly as possible without making errors. The hand used for the response is counterbalanced across subjects within the clinical constraints. After the response, the probe is turned off, and the subjects receive acoustic feedback regarding whether their response was correct or incorrect. Before initiating the next trial, the subjects were encouraged to blink and relax. The subjects perform 50 trials in one session, which last approximately 10 min. Trials with different set sizes are presented in a random order,with the single exception that a trial with an incorrect response is always followed by a trial with a set size of 4."
    task_url = task["Task URL"].values[0]
    return f"Task Name: {task_name}\nTask Description: {task_desc}\nTask URL: {task_url}"

In [937]:
def _get_related_publications(nix: nixio.File) -> List[str]:
    related_publications = nix.sections["General"].sections["Related publications"].props
    names_and_dois = zip(related_publications["Publication name"].values,
                         related_publications["Publication DOI"].values)
    return [f"{name.strip()} ({doi.strip()})" for (name, doi) in names_and_dois]

In [938]:
def write_subject(ctx: SessionContext):
    subject = ctx.nix.sections["Subject"].props
    age = subject["Age"].values[0]
    sex = subject["Sex"].values[0]
    ctx.nwb.subject = Subject(
        subject_id=f"{ctx.subject:02}",
        age=f"P{int(age)}Y",
        description=_get_subject_description(ctx),
        species="Homo sapiens",
        sex=_get_sex(sex),
    )

In [939]:
def _get_sex(raw: str) -> str:
    male = ["male", "m", "männlich", "maennlich", "mannlich", "männchen", "mannchen"]
    female = ["female", "f", "weiblich", "weibchen"]
    sex = raw.lower()
    return "M" if sex in male else "F" if sex in female else "O"

In [940]:
def _get_subject_description(ctx: SessionContext) -> str:
    subject = ctx.nix.sections["Subject"].props
    handedness = subject["Handedness"].values[0]
    pathology = subject["Pathology"].values[0]
    depth_electrodes = subject["Depth electrodes"].values[0]
    electrodes_in_soz = subject["Electrodes in seizure onset zone (SOZ)"].values[0]
    return f"Handedness: {handedness}\nPathology: {pathology}\nDepth electrodes: {depth_electrodes}\nElectrodes in seizure onset zone (SOZ): {electrodes_in_soz}"


In [941]:
def write_eeg_electrodes(ctx: SessionContext):
    nwb = ctx.nwb

    device = nwb.create_device(
        name="NicoletOne EEG System", manufacturer="Natus Medical Incorporated"
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name=f"eeg",
        description=f"EEG electrodes on scalp",
        device=device,
        location="Scalp",
    )

    electrodes = _get_eeg_electrodes(ctx)
    electrodes.apply(lambda row: _add_row_to_eeg_electrodes(nwb, electrode_group, row), axis=1)

In [942]:
def _add_row_to_eeg_electrodes(nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series):
    # got BESA map: +X is anterior, +Y is left, +Z is superior according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        location=row["label"],
        reference="Averaged mastoid channels",
        x=-row["x"],
        y=-row["z"],
        z=-row["y"],
        filtering="Passband, 0.5 to 5000 Hz",
    )

In [943]:
def _get_eeg_electrodes(ctx: SessionContext) -> pd.DataFrame:
    labels = _get_eeg_electrode_labels(ctx)
    locations = _get_eeg_electrode_locations(ctx)
    locations_array = np.ndarray(locations.shape)
    locations.read_direct(locations_array)
    df = pd.DataFrame(locations_array, columns=["x", "y", "z"])
    df.insert(0, "label", labels)
    return df.reset_index()

In [944]:
def _get_eeg_electrode_labels(ctx: SessionContext) -> List[str]:
    _assert_all_eeg_electrodes_have_same_labels(ctx)
    session_data = _get_session_data(ctx)
    return session_data.groups["Scalp EEG data"].data_arrays["Scalp_EEG_Data_Trial_01"].dimensions[0].labels

In [945]:
def _assert_all_eeg_electrodes_have_same_labels(ctx: SessionContext):
    data_arrays = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
    electrode_labels = [data_array.dimensions[0].labels for data_array in data_arrays]
    assert len(set(electrode_labels)) == 1

In [946]:
def _get_session_data(ctx: SessionContext) -> nixio.Block:
    return ctx.nix.blocks[f"Data_Subject_{ctx.subject:02}_Session_{ctx.session:02}"]

In [947]:
def _get_eeg_electrode_locations(ctx: SessionContext) -> nixio.DataArray:
    return _get_session_data(ctx).groups["Scalp EEG electrode information"].data_arrays[
        "Scalp_Electrode_EEGLAB_BESA_Coordinates"]

In [948]:
def write_ieeg_electrodes(ctx: SessionContext) -> ElectrodeGroup:
    nwb = ctx.nwb

    device = nwb.create_device(
        name="ATLAS Neurophysiology System", manufacturer="Neuralynx, Inc."
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name="ieeg",
        description=f"iEEG electrodes",
        device=device,
        location="Intracranial",
    )

    electrodes = _get_ieeg_electrodes(ctx)
    electrodes.apply(lambda row: _add_row_to_ieeg_electrodes(nwb, electrode_group, row), axis=1)
    return electrode_group

In [949]:
def _get_ieeg_electrodes(ctx: SessionContext) -> pd.DataFrame:
    labels = _get_ieeg_electrode_labels(ctx)
    locations = _get_ieeg_electrode_locations(ctx)
    locations_array = np.ndarray(locations.shape)
    locations.read_direct(locations_array)
    df = pd.DataFrame(locations_array, columns=["x", "y", "z"])
    df.insert(0, "label", labels)
    return df.reset_index()

In [950]:
def _get_ieeg_electrode_labels(ctx: SessionContext) -> List[str]:
    return _get_session_data(ctx).groups["iEEG electrode information"].data_arrays[
        "iEEG_Electrode_Manual_Entry"].dimensions[0].labels

In [951]:
def _get_ieeg_electrode_locations(ctx: SessionContext) -> nixio.DataArray:
    return _get_session_data(ctx).groups["iEEG electrode information"].data_arrays[
        "iEEG_Electrode_MNI_Coordinates"]

In [952]:
def _add_row_to_ieeg_electrodes(nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series):
    # Got MNI map: +X is right, +Y is anterior, +Z is superior according to <https://kathleenhupfeld.com/mni-template-coordinate-systems/>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        location=row["label"],
        reference="Common intracranial reference",
        x=-row["y"],
        y=-row["z"],
        z=row["x"],
        filtering="Passband, 0.5 to 1000 Hz",
    )

In [953]:
def write_eeg_measurements(ctx: SessionContext):
    nwb = ctx.nwb
    eeg_electrode_indices = list(range(_get_eeg_electrode_count(ctx)))
    eeg_table_region = nwb.create_electrode_table_region(
        region=eeg_electrode_indices,  # reference row indices 0 to N-1
        description="eeg electrodes",
    )
    trials = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
    for trial in trials:
        metadata = trial.dimensions[1]
        raw_electrical_series = ElectricalSeries(
            name=trial.name,
            data=trial[:].transpose(),
            electrodes=eeg_table_region,
            starting_time=metadata.offset,
            rate=1.0 / metadata.sampling_interval,
        )
        nwb.add_acquisition(raw_electrical_series)

In [954]:
def _get_eeg_electrode_count(ctx: SessionContext) -> int:
    return len(_get_eeg_electrode_labels(ctx))

In [955]:
def write_ieeg_measurements(ctx: SessionContext):
    nwb = ctx.nwb
    min_index = _get_eeg_electrode_count(ctx)
    ieeg_electrode_indices = list(range(min_index, min_index + _get_ieeg_electrode_count(ctx)))
    ieeg_table_region = nwb.create_electrode_table_region(
        region=ieeg_electrode_indices,  # reference row indices 0 to N-1
        description="ieeg electrodes",
    )
    trials = _get_session_data(ctx).groups["iEEG data"].data_arrays
    for trial in trials:
        metadata = trial.dimensions[1]
        raw_electrical_series = ElectricalSeries(
            name=trial.name,
            data=trial[:].transpose(),
            electrodes=ieeg_table_region,
            starting_time=metadata.offset,
            rate=1.0 / metadata.sampling_interval,
        )
        nwb.add_acquisition(raw_electrical_series)


In [956]:
def _get_ieeg_electrode_count(ctx: SessionContext) -> int:
    return len(_get_ieeg_electrode_labels(ctx))

In [957]:
def write_trial_data(ctx: SessionContext):
    nwb = ctx.nwb
    nwb.add_trial_column(name="trial_number",
                         description="The trial number")
    nwb.add_trial_column(name="set_size",
                         description="Number of letters shown during encoding period (4, 6, or 8 letters)")
    nwb.add_trial_column(name="probe_letter",
                         description="The letter presented to the participant during the retrieval period")
    nwb.add_trial_column(name="response",
                         description="The participant's answer to the question \"Was the letter at hand present in the encoding set?\"")
    nwb.add_trial_column(name="response_time",
                         description="The time at which the participant responded during the retrieval period")
    nwb.add_trial_column(name="solution",
                         description="The solution to the question \"Was the letter at hand present in the encoding set?\"")
    nwb.add_trial_column(name="artifact",
                         description="Whether the trial data was marked as an artifact by the experimenter")

    nix = ctx.nix
    trials = nix.sections["Session"].sections["Trial properties"].sections
    recordings = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
    for trial, recording in zip(trials, recordings):
        trial = trial.props
        metadata = recording.dimensions[1]
        start_time = metadata.offset
        stop_time = start_time + metadata.sampling_interval * recording.shape[1]
        response_time = trial["Response time"].values[0]
        # See https://gin.g-node.org/USZ_NCH/Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM/issues/2#issuecomment-3729
        response = str(trial["Response"].values[0])[1] == 1
        solution = int(trial["Match"].values[0]) == 1
        nwb.add_trial(
            start_time=start_time,
            stop_time=stop_time,
            trial_number=int(trial["Trial number"].values[0]),
            set_size=int(trial["Set size"].values[0]),
            probe_letter=str(trial["Probe letter"].values[0]),
            response=response,
            response_time=response_time,
            solution=solution,
            artifact=bool(trial["Artifact"].values[0]),
        )

In [958]:
def write_behavior(ctx: SessionContext):
    nwb = ctx.nwb
    behavior_module = nwb.create_processing_module(
        name="behavior", description="Data for all trials in this session."
    )
    nix = ctx.nix
    trials = nix.sections["Session"].sections["Trial properties"].sections
    for trial in trials:
        trial = trial.props
        trial_number = int(trial["Trial number"].values[0])
        name = f"trial_{trial_number:02}"
        # See https://gin.g-node.org/USZ_NCH/Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM/issues/2#issuecomment-3729
        data = [str(trial["Response"].values[0])[1] == 1]
        timestamps = [trial["Response time"].values[0]]

        time_series = TimeSeries(
            name=name,
            data=data,
            timestamps=timestamps,
            description="The participant's answer to the question \"Was the letter at hand present in the encoding set?\"",
            unit="n/a",  # Might as well use https://github.com/rly/ndx-events, but it's not built-in...
            continuity="instantaneous"
        )

        behavioral_events = BehavioralEvents(name=f"BehavioralEvents.{name}", time_series=time_series)

        behavior_module.add(behavioral_events)

In [959]:
_EVENT_RE = re.compile(r"Event_([a-zA-Z]+)_.*Trial_(\d\d)_.*")

In [960]:
def write_events(ctx: SessionContext):
    nwb = ctx.nwb
    session = _get_session_data(ctx)
    tags = session.groups["Trial events single tags spike times"].tags  # same as EEG and iEEG in this case
    tags_by_trial = [(_EVENT_RE.findall(tag.name)[0], tag.position) for tag in tags]
    trial_to_events = {}
    for (name, trial), position in tags_by_trial:
        trial_to_events.setdefault(trial, []).append((name, position[0]))
    _ensure_all_events_are_same_for_all_trials(trial_to_events)

    values = trial_to_events["01"]
    data = [value[0] for value in values if value[0] != "Response"]
    timestamps = [value[1] for value in values if value[0] != "Response"]

    time_series = TimeSeries(
        name="trial_events",
        data=data,
        timestamps=timestamps,
        unit="n/a",
        continuity="step",
        description="Each trial started with a fixation period ([−6, −5] s), followed by the stimulus ([−5, −3] s). The stimulus consisted of a set of eight consonants at the center of the screen. The middle four, six, or eight letters were the memory items, which determined the set size for the trial (4, 6, or 8, respectively). The outer positions were filled with “X,” which was never a memory item. After the stimulus, the letters disappeared from the screen, and the maintenance interval started ([−3, 0] s). A fixation square was shown throughout fixation, encoding, and maintenance. After maintenance, a probe was presented. The subjects responded with a button press to indicate whether the probe was part of the stimulus",
    )
    nwb.add_stimulus(time_series)


In [961]:
def _ensure_all_events_are_same_for_all_trials(trial_to_events: Dict[str, List[Tuple[str, float]]]):
    reference = trial_to_events["01"]
    for trial, events in trial_to_events.items():
        if len(events) != len(reference):
            raise ValueError(f"Trial {trial} has a different number of events than trial 01")
        for ((event, position), (reference_event, reference_position)) in zip(events, reference):
            if event == "Response":
                continue
            if event != reference_event:
                raise ValueError(f"Trial {trial} has a different event than trial 01")
            if position != reference_position:
                raise ValueError(f"Trial {trial} has a different event position than trial 01")

In [962]:
# Spike_Waveform_Unit_1_uAHL_2
_WAVEFORM_RE = re.compile(r"Spike_Waveform_Unit_(\d+)_(u[a-zA-Z]+)_(\d+)")
# Spike_Times_Unit_36_uPHR_1_Trial_16
_SPIKE_TIMES_RE = re.compile(r"Spike_Times_Unit_(\d+)_(u[a-zA-Z]+)_(\d+)_Trial_(\d+)")

In [963]:
def write_waveforms(ctx: SessionContext, ieeg_electrode_group: ElectrodeGroup):
    # Todo: These trials should globally be made a single timeline with invalid times inbetween
    nwb = ctx.nwb
    session = _get_session_data(ctx)
    waveforms = session.groups["Spike waveforms"].data_arrays
    spike_times = session.groups["Spike times"].data_arrays
    waveforms = [(_WAVEFORM_RE.findall(waveform.name)[0], waveform) for waveform in waveforms]
    spike_times = [(_SPIKE_TIMES_RE.findall(spike_time.name)[0], spike_time[:]) for spike_time in spike_times]
    unit_to_waveform = {int(unit): (electrode, channel, values) for ((unit, electrode, channel), values) in waveforms}
    unit_to_trial_to_spike_times = {}
    for ((unit, electrode, channel, trial), values) in spike_times:
        unit_to_trial_to_spike_times.setdefault(int(unit), {})[trial] = (electrode, channel, values)
    for unit, (electrode, channel, waveform) in unit_to_waveform.items():
        trial_to_spike_times = unit_to_trial_to_spike_times[unit]

        spike_times_for_trials = []
        for trial, (electrode_, channel_, spike_times) in trial_to_spike_times.items():
            assert electrode == electrode_
            assert channel == channel_
            spike_times_for_trials.append((trial, spike_times))
        spike_times_for_trials.sort(key=lambda x: x[0])
        spike_times_for_trials = [spike_times for _, spike_times in spike_times_for_trials]
        spike_times_for_trials = _untrialize_irregular_timestamps(spike_times_for_trials, -6)

        electrode_label = f"m{electrode}{channel}"
        data_point_count = waveform.shape[1]
        sample_interval = waveform.dimensions[1].sampling_interval
        offset = waveform.dimensions[1].offset
        min_time = 0 + offset
        max_time = min_time + data_point_count * sample_interval
        data = waveform[:]
        nwb.add_unit(
            id=int(unit),
            electrode_group=ieeg_electrode_group,
            electrodes=[electrode_label],
            waveform_mean=data[0],
            waveform_sd=data[1],
            obs_intervals=[(time + min_time, time + max_time) for time in spike_times_for_trials],
            spike_times=spike_times_for_trials,
        )

In [964]:
def _untrialize_irregular_timestamps(timestamps: List[List[float]], offset: float) -> List[float]:
    untrialized = []
    for trial, times in enumerate(timestamps):
        times = [time + offset + trial * _TRIAL_DURATION for time in times]
        untrialized.extend(times)
    return untrialized

In [965]:
_TRIAL_DURATION = 10.0 * 60.0 / 50.0  # Taken from paper: 10 minutes, 50 trials

In [966]:
def write_nwb(ctx: SessionContext):
    with NWBHDF5IO(f"subject{ctx.subject:02}_session{ctx.session:02}.nwb", "w") as io:
        io.write(ctx.nwb)

In [967]:
if __name__ == "__main__":
    context = main()
    #print(context.nwb.acquisition["Scalp_EEG_Data_Trial_01"])
    #print(context.nwb.electrodes.to_dataframe().to_string())

[-11.008170000000973, -10.291545000000042, -9.797795000005863, -8.325139750013477, -8.207671000011032, -8.069764750005561, -8.028233500008355, -7.553389750013594, -6.969109500016202, -6.638922000012826, -4.068235500017181, 0.4310807500005467, 0.5836120000021765, 1.6807682499929797, 2.5153922499885084, 3.68045474999235, 4.045486000002711, 6.107734999997774, 6.208109999992303, 6.917515249995631, 12.339955500006909, 13.621080500000971, 13.643454499993823, 13.667235750006512, 13.71839199999522, 13.751173249998828, 13.794517000002088, 13.86957949999487, 13.92757949999941, 13.93692325000302, 14.169704500003718, 14.53695449999941, 15.901829499998712, 16.963859750001575, 17.577797249992727, 24.360674750001635, 24.962486250005895, 25.1347987500194, 25.281361250017653, 28.0349227500119, 28.17086025001481, 28.201016500010155, 28.23961025000608, 28.52257900001132, 28.574641500003054, 29.169547750003403, 29.21439150000515, 29.259735250016092, 29.310641500007478, 29.31989150001027, 29.34570400000666

DELETE ANYTHING PYCHARM GENERATES AFTER THIS